In [211]:
import random as rn
import binascii
import hashlib
import math

In [212]:
p = 2**255 - 19
A = 486662

In [213]:
print(hex(p))

0x7fffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffed


In [214]:
c1 = (p + 3) // 8       # Integer arithmetic
c2 = pow(2, c1, p)
c3 = pow(2, (p-1) // 4, p)
c4 = (p - 5) // 8       # Integer arithmetic
c5 = pow(-486664, c1, p) * c3 % p #sqrt(-486664)

print(hex(c1))
print(hex(c2))
print(hex(c3))
print(hex(c4))
print(hex(c5))
print(c5 * c5 % p == -486664 % p)


0xffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffe
0x2b8324804fc1df0b2b4d00993dfbd7a72f431806ad2fe478c4ee1b274a0ea0b1
0x2b8324804fc1df0b2b4d00993dfbd7a72f431806ad2fe478c4ee1b274a0ea0b0
0xffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffd
0xf26edf460a006bbd27b08dc03fc4f7ec5a1d3d14b7d1a82cc6e04aaff457e06
True


In [215]:
hex(c3 * c3 % p)

'0x7fffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffec'

In [216]:
w = 0xffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffd
print(math.log2(w))
print(bin(w))

252.0
0b111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111101


In [235]:
def is_on_curve25519(x, y, z = 1):
    if z > 1:
        zinv = pow(z, p-2, p)
        x = x * zinv
        y = y * zinv
    return pow(y, 2, p) == (pow(x, 3, p) + A * pow(x, 2, p) + x) % p

In [218]:
def int2bytes(i) -> str:
    hex_string = '%x' % i
    n = len(hex_string)
    return binascii.unhexlify(hex_string.zfill(n + (n & 1)))

In [219]:
def print_bytes(s: str, b: bytes):
    print(s, "".join(format(x, "02x") for x in b))

In [220]:
def cmov(a, b, c):
    print("cmov: ", not c)
    if not c:
        return a
    return b
    
def sgn0(x):
    return x % 2

def inv0(x):
    return pow(x, p-2, p)

In [221]:
def sha512(s):
    return hashlib.sha512(s).digest()

In [222]:
def expand_message(MSG: bytes, DST: bytes) -> bytes:
    EXP_TAG = int2bytes(0x8000000000000000000000000000000000000000000000000000000000545301)
    MSG = EXP_TAG + MSG + int2bytes(0x20) + DST + int2bytes(0x1E)
    #print_bytes("to expand: ", MSG)
    return sha512(MSG)

In [223]:
def hash_to_field(MSG: str, DST: str) -> int:
    expanded = expand_message(MSG, DST)
    #print_bytes("expanded: ", expanded)
    return int.from_bytes(expanded, 'big') % p

In [252]:
def map_to_curve_elligator2_curve25519(u):
    tv1 = pow(u, 2, p)
    tv1 = 2 * tv1 % p
    xd = tv1 + 1 % p        # Nonzero: -1 is square (mod p), tv1 is not
    x1n = -A  % p             # x1 = x1n / xd = -A / (1 + 2 * u^2)
    tv2 = pow(xd, 2, p)
    gxd = tv2 * xd  % p       # gxd = xd^3
    gx1 = A * tv1  % p        # x1n + A * xd
    gx1 = gx1 * x1n  % p      # x1n^2 + A * x1n * xd
    gx1 = gx1 + tv2  % p      # x1n^2 + A * x1n * xd + xd^2
    gx1 = gx1 * x1n  % p      # x1n^3 + A * x1n^2 * xd + x1n * xd^2
    tv3 = pow(gxd, 2, p)
    tv2 = pow(tv3, 2, p)           # gxd^4
    tv3 = tv3 * gxd % p       # gxd^3
    tv3 = tv3 * gx1 % p       # gx1 * gxd^3
    tv2 = tv2 * tv3 % p       # gx1 * gxd^7
    print("tv2: ", hex(tv2))
    y11 = pow(tv2, c4, p)          # (gx1 * gxd^7)^((p - 5) / 8)
    print("y11: ", hex(y11))
    y11 = y11 * tv3  % p       # gx1 * gxd^3 * (gx1 * gxd^7)^((p - 5) / 8)
    y12 = y11 * c3  % p
    tv2 = pow(y11, 2, p)
    tv2 = tv2 * gxd  % p
    e1 = tv2 == gx1
    y1 = cmov(y12, y11, e1)  # If g(x1) is square, this is its sqrt
    x2n = x1n * tv1  % p          # x2 = x2n / xd = 2 * u^2 * x1n / xd
    y21 = y11 * u  % p
    y21 = y21 * c2 % p
    y22 = y21 * c3 % p
    gx2 = gx1 * tv1  % p          # g(x2) = gx2 / gxd = 2 * u^2 * g(x1)
    tv2 = pow(y21, 2, p)
    tv2 = tv2 * gxd % p
    e2 = tv2 == gx2
    y2 = cmov(y22, y21, e2)  # If g(x2) is square, this is its sqrt
    tv2 = pow(y1, 2, p)
    tv2 = tv2 * gxd % p
    e3 = tv2 == gx1
    xn = cmov(x2n, x1n, e3)  # If e3, x = x1, else x = x2
    y = cmov(y2, y1, e3)    # If e3, y = y1, else y = y2
    e4 = sgn0(y) == 1        # Fix sign of y
    y = cmov(y, -y % p, e3 ^ e4)
    return (xn, xd, y, 1)

In [236]:
def point_generate_curve25519(DST: str, rng: int):
    m = int2bytes(rng)
    #print("rng: ", hex(rng))
    #print_bytes("m: ", m)

    u = hash_to_field(m, DST)

    #print("u: ", hex(u))

    xn, xd, yn, yd = map_to_curve_elligator2_curve25519(u)

    #print("xn", hex(xn))

    x = xn * inv0(xd) % p
    y = yn * inv0(yd) % p

    x = xn
    z = xd
    y = yn * z % p

    return x, y, z

In [226]:
rng = rn.randint(0, 2**256 - 1)

In [238]:
DST = int2bytes(0x54535F53504543545F4453540000000000000000000000000000000000D8)

x, y, z = point_generate_curve25519(DST, rng)

print("x: ", hex(x))
print("y: ", hex(y))
print("z: ", hex(z))
print(is_on_curve25519(x, y, z))


x:  0x198156d1cad3bbb320dd2c007b3131907865fd3d7372ad1724afca1d8fcf854f
y:  0x1d6aef266cd9bbea43dace0bcd07f723d323ea5eb4ecdc5f8b907d5e59279a96
z:  0x169dad2744d96e4d416101540114ce150134516388346edfa4a6d9cf2166b425
True


In [260]:
def point_generate_curve25519(DST: str, rng: int):
    m = int2bytes(rng)
    u = hash_to_field(m, DST)
    xn, xd, yn, yd = map_to_curve_elligator2_curve25519(u)
    x = xn * inv0(xd) % p
    y = yn * inv0(yd) % p
    x = xn
    z = xd
    y = yn * z % p
    return x, y, z

def map_to_edwards(xMn, xMd, yMn, yMd):
    xn = xMn * yMd % p
    xn = xn * c5 % p
    xd = xMd * yMn % p    # xn / xd = c1 * xM / yM
    yn = xMn - xMd % p
    yd = xMn + xMd % p   # (n / d - 1) / (n / d + 1) = (n - d) / (n + d)
    tv1 = xd * yd % p
    e = tv1 == 0
    if e:
        return (0, 1, 1, 1)
    return (xn, xd, yn, yd)

def point_generate_ed25519(DST: str, rng: int):
    m = int2bytes(rng)
    u = hash_to_field(m, DST)
    xMn, xMd, yMn, yMd = map_to_curve_elligator2_curve25519(u)
    xn, xd, yn, yd = map_to_edwards(xMn, xMd, yMn, yMd)
    xq = xn * pow(xd, p-2, p)
    yq = yn * pow(yd, p-2, p)
    x = xn * yd % p
    y = yn * xd % p
    z = xd * yd % p
    return (x, y, z)

def is_on_ed25519(x, y, z = 1):
    if z > 1:
        zinv = pow(z, p-2, p)
        x = x * zinv
        y = y * zinv
    return ( -pow(x, 2, p) + pow(y, 2, p) ) % p == 1 + ( ((-121665)*pow(121666, p-2, p)) * pow(x, 2, p) * pow(y, 2, p) ) % p 

In [261]:
DST = int2bytes(0x54535F53504543545F4453540000000000000000000000000000000000D8)

for r in [rn.randint(0, 2**256 - 1) for i in range(1000)]:
    x, y, z = point_generate_ed25519(DST, r)
    #print("x: ", hex(x))
    #print("y: ", hex(y))
    if not is_on_ed25519(x, y, z):
        print("fail")
        print(hex(r))
        break

In [230]:
u = rn.randint(0, p-1)
print(hex(u))

0x92e9668b0661bf4bb0567ea732ad1664a7eb0e9a3b5995ab44d07ed3fd8b6fb


In [231]:
xn, xd, yn, yd = map_to_curve_elligator2_curve25519(u)

x = xn * inv0(xd) % p
y = yn * inv0(yd) % p

print(hex(x))
print(hex(y))

0xad6231d912a03e9725fc2bee70d41ae5cb862ffa78092eb83781a8cedf5d91b
0x2a38c389719f83254cf80b9eb1e0bd4befd955212f36b9d79c392dfcc4afb2fc


In [232]:
pow(y, 2, p) == (pow(x, 3, p) + A * pow(x, 2, p) + x) % p

True

In [233]:
for i in range(1000):
    u = rn.randint(0, p-1)
    xn, xd, yn, yd = map_to_curve_elligator2_curve25519(u)
    x = xn * inv0(xd) % p
    y = yn * inv0(yd) % p
    if not is_on_curve25519(x, y):
        print("Failed on ", hex(u))
        break
print("END")

END
